# Part 3: preparation for next time - wave spectrum

In [ ]:
# compute PSD and extract peak wavenumer
fig, ax = plt.subplots()
#ax.set_ylim([5e-6, 1e5])


fps=sample_rate  # sampling frequency - in minutes??? change.
# compute spectrum
fft = np.fft.rfft(z, n=len(z)) # note: mean already removed from signal
freq = np.fft.rfftfreq(len(z), d=1/fps)

kind = 'PSD'
if kind == 'amplitude':
    r_fft = np.abs(fft)
if kind == 'power':
    r_fft = np.abs(fft)**2
if kind == 'PSD':
    r_fft = (np.abs(fft)**2)*2/fps/len(fft)  # normalize by frequency bin width
# r_fft_norm = r_fft/len(r_fft) #? check
# plot spectrum:
ax.plot(freq, r_fft, linewidth=.5)  #CHECK NORMALIZATION - im doing something incorrect 

ax.set_xlabel('f (Hz)')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylabel('{}'.format(kind))
ax.set_title('wave spectrum: buoy 5 min') #.format(wave,wind))

ax.set_xlim([0.02,1])
ax.set_ylim([0.001,100])

plt.show()

# smooth spectrum by averaging signal in windows:
#freq_win, fft_win = sig.welch(z, fps, window='hann', scaling='density', nperseg=1024) 
#ax.plot(freq_win, fft_win, 'C1', linewidth=1.5)  # window correction factor 1.5?

#find peaks of smoothed spectrum and estimate peak frequency
# peaks_i = sig.find_peaks(fft_win)
# f_p = freq_win[peaks_i[0][np.where(fft_win[peaks_i[0]] == fft_win[peaks_i[0]].max())[0][0]]]
# print('peak frequency = {}'.format(f_p))

# filter and smooth
# test also with different input signal window
# integrate to find a significant wave height estimate